In [4]:
from skyfield.api import load
import numpy as np

# Load the JPL ephemeris
eph = load('de440s.bsp')
# load time
ts = load.timescale()

# Define the time of interest
t = ts.utc(2023, 8, 1, 15, 0, 0)

# check time
print(t.utc_strftime('On %Y %b %d at %H:%M:%S'))

# Get the position of the Earth at the given time
earth = eph['Earth'].at(t)

# Get the position vector of the Earth in the J2000 reference frame
earth_pos = earth.position.km

On 2023 Aug 01 at 15:00:00


In [2]:
# Get the position of the Sun relative to the Earth at the given time
sun = eph['Sun'].at(t)

# Get the position vector of the Sun relative to the Earth in the J2000 reference frame
sun_pos = sun.position.km - earth_pos

In [6]:
# Compute the position vector from the Earth to the satellite
r_sat = np.array([-346, 8265, 4680])  # km
r_earth = earth.position.km
r_vec = r_sat - r_earth

# Compute the distance from the Earth to the satellite
r = np.linalg.norm(r_vec)

# Compute the apparent radius of the Earth as seen from the satellite
R_e = 6378.137  # km (mean equatorial radius of the Earth)
rho = np.arcsin(R_e/r)

# Compute the angle between the satellite position vector and the vector to the center of the Earth
theta = np.arccos(np.dot(r_vec, -r_earth)/(r*np.linalg.norm(-r_earth)))

# Compute the distance from the satellite to the edge of the Earth's shadow
s = R_e*np.tan(theta+rho)

# Determine whether the satellite is in the shadow or not
if s >= r:
    print("The satellite has a clear view of the Sun.")
else:
    # Compute the penumbral and umbral radii
    r_penum = np.arcsin(R_e/(R_e+s))*r
    r_umbral = np.arcsin(R_e/(R_e-s))*r
    if r >= r_umbral:
        print("The satellite is in the penumbra.")
    elif r <= r_penum:
        print("The satellite is in the umbra.")

The satellite is in the umbra.


/var/folders/cf/x6zpxdwn2dg40q0c0r418f7c0000gp/T/ipykernel_3645/3871601537.py:25: RuntimeWarning: invalid value encountered in arcsin
  r_umbral = np.arcsin(R_e/(R_e-s))*r
